In [25]:
import os
import re
import numpy as np
import tifffile
from osgeo import gdal
# from skimage import io 
# from sklearn.model_selection import train_test_split
import cv2
import  json


In [26]:
def img_transposition(img):

    bands=[0, 1, 2, 7, 8, 9]
    # img1=np.transpose(img[0,bands,:,:],(1,2,0))
    # img2=np.transpose(img[1,bands,:,:],(1,2,0))
    # img3=np.transpose(img[2,bands,:,:],(1,2,0))
    img_shape=img.shape
    params=np.zeros(6)
    new_img_time_series = np.empty((0, img_shape[1], img_shape[2], img_shape[3]), dtype=img.dtype)
    for i in range(img_shape[1]):

        params[0]=np.mean(img[i,0,:,:])
        params[1]=np.mean(img[i,1,:,:])
        params[2]=np.mean(img[i,2,:,:])
        params[3]=np.median(img[i,0,:,:])
        params[4]=np.median(img[i,1,:,:])
        params[5]=np.median(img[i,2,:,:])
        
        if np.all(params[0:3] < 4000) and np.all(params[3:6] > 200):
            new_img_time_series = np.vstack((new_img_time_series, img[i,:,:,:][np.newaxis]))



    new_img_time_series = np.array(new_img_time_series)
    new_shape=new_img_time_series.shape

    # print(new_shape)
    middle_index = new_shape[0] // 2
    img1 = np.transpose(new_img_time_series[0, bands, :, :], (1, 2, 0))
    img2 = np.transpose(new_img_time_series[middle_index, bands, :, :], (1, 2, 0))
    img3 = np.transpose(new_img_time_series[-1, bands, :, :], (1, 2, 0))


    # merged_img=np.concatenate([img1,img2,img3], axis=2)
    merged_img =np.dstack((img1,img2,img3))
    # transposed_img = np.transpose(merged_img, (2,0,1))
    # print(transposed_img.shape)
    return merged_img

In [27]:
def normalize_image(image, fold_values, id_patch_to_fold,parcel_id_nb):
        bands=[0, 1, 2, 7, 8, 9]
        normalized_image = np.zeros_like(image, dtype=np.float32)
        for channel in range(6):  # 3x6 kanałów
            # print(image[0,0:9,0:9])            
            normalized_image[:, :, channel] = (image[:, :, channel] - fold_values[id_patch_to_fold[parcel_id_nb]]['mean'][bands[channel]]) / fold_values[id_patch_to_fold[parcel_id_nb]]['std'][bands[channel]]
            normalized_image[:, :, channel+6] = (image[:, :, channel+6] - fold_values[id_patch_to_fold[parcel_id_nb]]['mean'][bands[channel]]) / fold_values[id_patch_to_fold[parcel_id_nb]]['std'][bands[channel]]
            normalized_image[:, :, channel+12] = (image[:, :, channel+12] - fold_values[id_patch_to_fold[parcel_id_nb]]['mean'][bands[channel]]) / fold_values[id_patch_to_fold[parcel_id_nb]]['std'][bands[channel]]
            # print(normalized_image[0,0:9,0:9]) 
            # dupamatrix=normalized_image[0,0:9,0:9]
            
        return normalized_image    

In [28]:
def resize_img(image, size):

    if len(image.shape)==2:
        height, width=image.shape
        channels=1
    elif len(image.shape)==3:
        height,width, channels=image.shape
    else:
        raise ValueError("Unsuported image shape")

    padded_img=np.zeros((size[0],size[1], channels),dtype=np.float32)
    y_offset=(size[0]-height) // 2
    x_offset=(size[0]-width) // 2


    if channels==1:
        padded_img[y_offset:y_offset+height,x_offset:x_offset+width,0]=image
    else:
        padded_img[y_offset:y_offset+height,x_offset:x_offset+width,:]=image

    return padded_img


In [29]:
def proces_img(dataset_path, mask_path, save_path, prefix, fold_values, id_patch_to_fold,validation_fold_nb,upscaling_img):
    old_save=save_path
    training_i=0
    validation_i=0
    processed_files_training = []
    processed_files_validation = []
    size=(224,224)
    val_save_path= os.path.join(save_path, 'validation_chips')
    train_save_patch=os.path.join(save_path, 'training_chips')

    for file_name in os.listdir(dataset_path):
        
        if file_name.endswith(".npy"):

            match = re.search(r'S2_(\d{2})(\d{3}).npy', file_name)
            yyy = match.group(1)
            xxx = match.group(2)
            mask_pth=os.path.join(mask_path,f"TARGET_{yyy}{xxx}.npy")
            parcel_id_nb=int(yyy + xxx)



            if match and os.path.exists(mask_pth):
                # print("dupa")
                if id_patch_to_fold[parcel_id_nb]==validation_fold_nb:
                    save_path =val_save_path
                else:
                    save_path = train_save_patch

                


                data=np.load(os.path.join(dataset_path,file_name))

                new_file_name=f"{prefix}_{xxx}_{yyy}_merged.tif"
                # np.save(os.path.join(save_path, new_file_name), data)
                img1=img_transposition(data)
                img2=normalize_image(img1,fold_values,id_patch_to_fold,parcel_id_nb)
                # img2=img1

                
                if upscaling_img:
                    imgscal=cv2.resize(img2,size,interpolation=cv2.INTER_LANCZOS4)
                else:
                    imgscal=resize_img(img2,size)
                # print(imgscal.shape)
                # tifffile.imwrite(os.path.join(save_path, new_file_name), img1)
                # cv2.imwrite(os.path.join(save_path, new_file_name + ".tif"), img1.astype(np.int16))

                driver = gdal.GetDriverByName('GTiff')  # Wybór sterownika dla formatu pliku TIFF
                image_height, image_width, num_bands = imgscal.shape  # Wymiary obrazu i liczba kanałów

                # Tworzenie nowego pliku rastrowego
                output_path = os.path.join(save_path, new_file_name)
   
                # output_dataset = driver.Create(output_path, image_width, image_height, num_bands, gdal.GDT_Float32)  # Tworzenie nowego pliku TIFF z 18 kanałami
                output_dataset = driver.Create(output_path, image_width, image_height, num_bands, gdal.GDT_Float32)  # Tworzenie nowego pliku TIFF z 18 kanałami

                


                # Zapisywanie danych obrazu do pliku rastrowego
                for band_idx in range(num_bands):
                    output_dataset.GetRasterBand(band_idx + 1).WriteArray(imgscal[:, :, band_idx].astype(np.float32))  # Zapisywanie danych do kolejnych kanałów


                # tifffile.imwrite(os.path.join(save_path, new_file_name), img1,metadata=tags)
                mask=np.load(mask_pth)
                
                extracted_layer=mask[0,:,:]



                reduced_mask = extracted_layer.copy()
                # print(reduced_mask.shape)
                mask_13_to_18 = (extracted_layer >= 13) & (extracted_layer <= 18)
                mask_13_to_18_indices = np.where(mask_13_to_18)
                reduced_mask[mask_13_to_18_indices] = 12
                reduced_mask[np.where(extracted_layer==19)]=0
                reduced_mask[np.where(extracted_layer==0)]=13

                reduced_mask[np.where(extracted_layer<0)]=13

                

                
                # reduced_mask(extracted_layer == 19) = 13
                new_mask_name=f"{prefix}_{xxx}_{yyy}.mask.tif"
                # np.save(os.path.join(save_path, new_mask_name), extracted_layer) 
                
                if upscaling_img:
                    maskscal=cv2.resize(reduced_mask,size,interpolation=cv2.INTER_NEAREST)
                else:

                    maskscal=resize_img(reduced_mask,size)

                maskscal_str=maskscal.astype(np.int16)

                tifffile.imwrite(os.path.join(save_path, new_mask_name), maskscal_str)  

                if id_patch_to_fold[parcel_id_nb]==validation_fold_nb:
                    processed_files_validation.append(f"{prefix}_{xxx}_{yyy}")
                    validation_i+=1
                else:
                    processed_files_training.append(f"{prefix}_{xxx}_{yyy}")
                    training_i+=1
                
                
                
    with open(os.path.join(old_save,"validation_data.txt"), 'w') as txt_file:
        for file_name in processed_files_validation:
            txt_file.write(file_name + '\n')
    with open(os.path.join(old_save,"training_data.txt"), 'w') as txt_file:
        for file_name in processed_files_training:
            txt_file.write(file_name + '\n')
    print(f"generated training images: {training_i}")
    print(f"generated validation images: {validation_i}")
    return 

In [30]:
# paths

dataset_path=r'F:\mgr_datasets\PASTIS\DATA_S2'
mask_path=r'F:\mgr_datasets\PASTIS\ANNOTATIONS'
save_path=r'F:\mgr_datasets\multismall'
# val_set_path=r'F:\mgr_datasets\multimoj\validation_chips'
json_path=r'F:\mgr_datasets\PASTIS\NORM_S2_patch.json'
geojson_path=r'F:\mgr_datasets\PASTIS\metadata.geojson'

In [31]:
with open(json_path, 'r') as f:
    folds_data = json.load(f)

fold_values = {}
i=1
for fold_name, fold_data in folds_data.items():
    mean_values = fold_data["mean"]
    std_values = fold_data["std"]
    
    # Dodanie wartości dla aktualnego foldu do słownika
    fold_values[i] = {"mean": mean_values, "std": std_values}
    i+=1

In [32]:
with open(geojson_path, 'r') as f:
    folds_data = json.load(f)

In [33]:
id_patch_to_fold = {}
for feature in folds_data['features']:
    properties = feature['properties']
    if 'ID_PATCH' in properties and 'Fold' in properties:
        id_patch = properties['ID_PATCH']
        fold = properties['Fold']
        id_patch_to_fold[id_patch] = fold

In [34]:
dataset_files=os.listdir(dataset_path)
# train_files, val_files = train_test_split(dataset_files, test_size=0.2, random_state=42)



In [35]:
training_save_path = os.path.join(save_path, "training_chips")
validation_save_path = os.path.join(save_path, "validation_chips")
os.makedirs(training_save_path, exist_ok=True)
os.makedirs(validation_save_path, exist_ok=True)


In [36]:
upscaling_img=False
validation_fold_nb=5

proces_img(dataset_path, mask_path, save_path, 'chip', fold_values,id_patch_to_fold,validation_fold_nb,upscaling_img)

# Wywołaj funkcje przetwarzające obrazy i masek dla danych walidacyjnych

# proces_img(dataset_path, mask_path, val_set_path, 'chip',"validation_data.txt",fold_values,id_patch_to_fold)

generated training images: 1937
generated validation images: 496


In [37]:
img_pth=r'F:\mgr_datasets\multimoj\training_chips\chip_001_10_merged.tif'
img=tifffile.imread(img_pth)

In [38]:
print(fold_values[1]['mean'][0])

1165.9398193359375


In [39]:
os.path.exists(mask_path)
print(mask_path)

F:\mgr_datasets\PASTIS\ANNOTATIONS


In [40]:
# driver = gdal.GetDriverByName('GTiff')
# output_dataset = driver.Create(r'F:\mgr_datasets\multismall\training_chips\chip_000_10_merged.tif', 224, 224, 18)